In [18]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score, StratifiedKFold, learning_curve, KFold
from sklearn.preprocessing import MinMaxScaler 
from sklearn.metrics import r2_score, mean_squared_error

In [7]:
bikes_test = pd.read_excel('bike_test.xlsx')
bikes_train = pd.read_excel('bike_train.xlsx')

In [8]:
bikes_train_temp = bikes_train.copy()
bikes_train_temp = bikes_train_temp.drop(["casual","registered","dteday"], axis=1)

In [9]:
y = bikes_train_temp["cnt"]
X = bikes_train_temp.drop("cnt", axis=1)

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size= 0.2, random_state=42)

In [11]:
xg_model = XGBRegressor()
xg_scores = cross_val_score(xg_model, X, y, scoring="neg_mean_squared_error", cv=10)

In [12]:
rmse = np.sqrt(-xg_scores)
rmse

array([31.61940745, 32.83884531, 45.47874768, 38.7529446 , 49.67146123,
       42.36170521, 52.00064884, 49.9246173 , 49.84449526, 68.09513171])

In [13]:
np.mean(rmse).round()

46.0

In [14]:
xg_model.fit(X_train, y_train)
xg_model_prediction = xg_model.predict(X_test)

In [15]:
print('R2 Score: ', r2_score(y_test, xg_model_prediction)*100)
print('RMSE: ', np.sqrt(mean_squared_error(y_test, xg_model_prediction)))

R2 Score:  95.21793754252931
RMSE:  32.86705739344138


In [21]:
bikes_test_temp = bikes_test.copy()

In [22]:
bikes_test_temp = bikes_test_temp.drop('dteday', axis=1)

In [ ]:
# xg_model_prediction_proper = xg_model.predict(bikes_test_temp)

In [ ]:
# len(xg_model_prediction_proper)

In [19]:
# Initialize our model
xg_boost = XGBRegressor( learning_rate=0.01,
                         n_estimators=6000,
                         max_depth=4, min_child_weight=1,
                         gamma=0.6, subsample=0.7,
                         colsample_bytree=0.2,
                         objective='reg:linear', nthread=-1,
                         scale_pos_weight=1, seed=27,
                         reg_alpha=0.00006
                       )

# Perform cross-validation to see how well our model does 
kf = KFold(n_splits=5)
y_pred = cross_val_score(xg_boost, X, y, cv=kf, n_jobs=-1)
y_pred.mean()

0.7144160001941389

In [20]:
xg_boost.fit(X,y)

[22:05:22] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.2,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0.6, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.01, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=4, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=6000,
             n_jobs=-1, nthread=-1, num_parallel_tree=1, objective='reg:linear',
             predictor='auto', random_state=27, ...)

In [23]:
xgb_pred = xg_boost.predict(bikes_test_temp)

In [24]:
xgb_pred

array([ 38.71084 ,  29.371586,  80.00939 , ..., 120.85918 ,  76.58758 ,
        48.825077], dtype=float32)

In [25]:
len(xgb_pred)

5380

In [26]:
prediction = pd.DataFrame()
prediction['pred'] = xgb_pred
prediction

,pred
0,38.710838
1,29.371586
2,80.009392
3,182.296661
4,302.067719
...,...
5375,224.485123
5376,150.490524
5377,120.859177
5378,76.587578


In [27]:
prediction.to_csv('KevinG91.csv', index= False)